In [1]:
from pydantic import BaseModel
from bs4 import BeautifulSoup
import requests
import rich
from pathlib import Path
import os
import re
import pandas as pd
from typing import Any
import importlib
import tqdm
from ruamel.yaml import YAML

while not (Path.cwd() / "pyproject.toml").exists():
    os.chdir("..")

from twfy_votes.helpers.data.models import data_to_yaml
from twfy_votes.apps.policies import models
from twfy_votes.apps.core.db import load_common_core

importlib.reload(models)

policy_dir = Path("data", "policies")

duck_core = await load_common_core()
duck = await duck_core.child_query()

In [2]:
importlib.reload(models)

PolicyReport = models.PolicyReport
PolicyStatus = models.PolicyStatus
IssueType = models.IssueType
reports = await PolicyReport.fetch_multiple(
    [PolicyStatus.ACTIVE, PolicyStatus.CANDIDATE]
)
reports = [x for x in reports if IssueType.STRONG_WITHOUT_POWER in x.division_issues]

yaml = YAML()
yaml.default_flow_style = False

In [6]:
def fix_policy(report: PolicyReport):
    change_count = 0
    policy_data = yaml.load(policy_dir / f"{report.policy.id}.yml")
    issue_divisions = report.division_issues[IssueType.STRONG_WITHOUT_POWER]
    keys = [x.key for x in issue_divisions]

    new_division_links = []
    for link in policy_data["division_links"]:
        decision = link["decision"]
        str_date = decision["date"].strftime("%Y-%m-%d")
        key = "-".join(
            [decision["chamber_slug"], str_date, str(decision["division_number"])]
        )
        if key in keys:
            link["strength"] = "weak"
            change_count += 1
        new_division_links.append(link)

    policy_data["division_links"] = new_division_links

    print(f"Changed {change_count} links for {report.policy.id}")

    any_strong = any([x["strength"] == "strong" for x in policy_data["division_links"]])

    if not any_strong:
        policy_data["status"] = "retired"
        print(f"Setting {report.policy.id} to retired")

    data_to_yaml(policy_data, policy_dir / f"{report.policy.id}.yml")
    return change_count


total_count = 0
for report in reports:
    total_count += fix_policy(report)

print(f"Total changes: {total_count}")

Changed 3 links for 810
Changed 1 links for 837
Changed 4 links for 975
Changed 4 links for 984
Changed 3 links for 1030
Changed 7 links for 1065
Changed 1 links for 1074
Changed 1 links for 1079
Changed 1 links for 1084
Changed 2 links for 1087
Changed 1 links for 1105
Changed 9 links for 1110
Changed 1 links for 1136
Changed 7 links for 6667
Changed 17 links for 6670
Changed 3 links for 6671
Changed 6 links for 6672
Changed 6 links for 6673
Changed 2 links for 6676
Changed 2 links for 6677
Changed 3 links for 6681
Changed 2 links for 6682
Changed 10 links for 6685
Changed 1 links for 6687
Changed 2 links for 6690
Changed 4 links for 6692
Changed 2 links for 6693
Changed 7 links for 6695
Changed 1 links for 6696
Changed 1 links for 6697
Changed 1 links for 6698
Changed 9 links for 6702
Changed 1 links for 6703
Changed 2 links for 6705
Changed 2 links for 6706
Changed 2 links for 6709
Changed 4 links for 6710
Changed 2 links for 6711
Changed 1 links for 6718
Changed 1 links for 6720
Ch